In [ ]:
!pip install langchain langchain-openai faiss-cpu pypdf sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [50]:
!pip install pdfplumber

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [182]:
# STEP 2: Imports
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os, re, pdfplumber
from collections import deque

In [183]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [184]:
pdf_path = "/content/drive/MyDrive/10MS Dataset/HSC26-Bangla1st-Paper.pdf"
print(os.path.exists(pdf_path))
# STEP 5: Load PDF
#loader = PyPDFLoader(pdf_path)
#docs = loader.load()

True


In [185]:
import re
import pdfplumber

def extract_and_clean(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text += page_text + "\n"

    # Remove English letters, numbers, common symbols
    text = re.sub(r"[A-Za-z0-9]", "", text)
    text = re.sub(r"[“”\"'•©®]", "", text)

    # Remove page numbers (standalone digits on lines)
    text = re.sub(r"^\s*\d+\s*$", "", text, flags=re.MULTILINE)

    # Remove excessive punctuation (keep Bangla sentence markers)
    text = re.sub(r"[^।!?০-৯\u0980-\u09FF\s]", " ", text)

    # Fix broken Bangla conjuncts (line breaks mid-word)
    text = re.sub(r"(\S)-\n(\S)", r"\1\2", text)  # handle hyphen splits
    text = text.replace("\n", " ")  # merge lines

    # Normalize spaces
    text = re.sub(r"\s+", " ", text)

    # Remove invisible characters
    text = re.sub(r"[\u200c\u200d]", "", text)  # ZWNJ, ZWJ

    return text.strip()

# Usage
raw_text = extract_and_clean(pdf_path)
print(raw_text[:1000])

িাাং া ১ম পত্র আল ািয রিষয় অপরিরিতা অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা র্িখনফল র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্। তৎকালীনসমাি সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্। তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্। তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্। নািী যকামল ঠিক র্কন্তু দব্ু লি ন কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত পািকব্। মানুষ আিা র্নক যব্কঁ চ থাকক অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি জ্ঞাানলাভকিকব্। প্রাক মূলযা ন ১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক ডাক্তার্ি খ ওকালর্ত গ মাস্টার্ি ঘ ব্যব্সা ২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ তাি ক প্রর্তপজি খ প্রভাব্ গ র্ব্চক্ষণতা ঘ কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও। র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্নত যব্াধ ককি র্ব্ক ি আক

In [186]:
import re

def bangla_text_quality_report(text):
    total_chars = len(text)
    bangla_chars = len(re.findall(r'[\u0980-\u09FF]', text))
    bangla_ratio = (bangla_chars / total_chars) * 100 if total_chars > 0 else 0

    # Sentence count (Bangla sentences usually end with "।")
    sentences = re.split(r'[।!?]', text)
    avg_sentence_length = sum(len(s) for s in sentences if s.strip()) / max(len(sentences), 1)

    # Check for leftover English words/numbers
    english_segments = re.findall(r'[A-Za-z0-9]+', text)

    print(f"Total characters: {total_chars}")
    print(f"Bangla characters: {bangla_chars}")
    print(f"Bangla ratio: {bangla_ratio:.2f}%")
    print(f"Estimated sentence count: {len(sentences)}")
    print(f"Average sentence length: {avg_sentence_length:.1f} chars")
    print(f"English/number segments found: {len(english_segments)}")

    return {
        "total_chars": total_chars,
        "bangla_ratio": bangla_ratio,
        "avg_sentence_length": avg_sentence_length,
        "english_segments": english_segments[:10]  # show first few
    }

# Example usage:
report = bangla_text_quality_report(raw_text)
print(report)

Total characters: 79715
Bangla characters: 67006
Bangla ratio: 84.06%
Estimated sentence count: 1173
Average sentence length: 67.0 chars
English/number segments found: 0
{'total_chars': 79715, 'bangla_ratio': 84.05695289468731, 'avg_sentence_length': 66.95907928388746, 'english_segments': []}


In [187]:
unique_chars = sorted(set(raw_text))
print(unique_chars)

[' ', '!', '?', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৌ', '্', 'ৎ', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ']


In [188]:
from collections import Counter

char_freq = Counter(raw_text)
for char, freq in char_freq.most_common(100):
    print(f"'{char}' : {freq}")

' ' : 11537
'া' : 7509
'ক' : 7481
'্' : 7360
'ি' : 6924
'র' : 3747
'ন' : 3246
'ত' : 2458
'ব' : 2337
'ম' : 2140
'য' : 2079
'ল' : 1716
'প' : 1608
'স' : 1470
'ু' : 1383
'দ' : 1257
'ে' : 1150
'গ' : 996
'হ' : 966
'।' : 961
'ই' : 728
'ট' : 693
'ী' : 678
'অ' : 626
'খ' : 620
'আ' : 600
'চ' : 551
'জ' : 482
'এ' : 452
'ভ' : 449
'ষ' : 418
'থ' : 413
'ণ' : 375
'উ' : 353
'ও' : 325
'ঘ' : 306
'়' : 291
'ধ' : 282
'১' : 205
'?' : 204
'২' : 191
'ং' : 164
'ঠ' : 162
'ঁ' : 153
'শ' : 153
'ূ' : 147
'ৃ' : 133
'ফ' : 111
'৯' : 97
'ঙ' : 94
'৪' : 92
'৩' : 91
'০' : 85
'৭' : 79
'৬' : 75
'ৌ' : 73
'৮' : 70
'৫' : 63
'ড' : 61
'ঞ' : 58
'ঝ' : 56
'য়' : 50
'ৎ' : 34
'ঐ' : 12
'ঢ' : 12
'ঃ' : 9
'!' : 7
'ঋ' : 2
'ঢ়' : 2
'ঈ' : 2
'ৰ' : 1


In [190]:
splitter = RecursiveCharacterTextSplitter(
    #separators=["\n\n", "\n", "।", "."],
    chunk_size=800,
    chunk_overlap=100
)
chunks = splitter.split_text(raw_text)
print("Total chunks:", len(chunks))
print(chunks[0])

Total chunks: 114
িাাং া ১ম পত্র আল ািয রিষয় অপরিরিতা অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা র্িখনফল র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ সংকটসম্পককি ধািণালাভকিকব্। তৎকালীনসমাি সভযতাওমানব্তািঅব্মাননাসম্পককি িানকতপািকব্। তৎকালীনসমাকিিপণপ্রথািকুপ্রভাব্সম্পককি িানকতপািকব্। তৎকাকলসমাকিভদ্রকলাককিস্বভাব্বব্র্িষ্ট্যসম্পককি জ্ঞাানলাভকিকব্। নািী যকামল ঠিক র্কন্তু দব্ু লি ন কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত পািকব্। মানুষ আিা র্নক যব্কঁ চ থাকক অনপু কমি দষ্ট্ৃ াকন্ত মানব্িীব্কনি এই র্চিন্তন সতযদিিন সম্পককি জ্ঞাানলাভকিকব্। প্রাক মূলযা ন ১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক ডাক্তার্ি খ ওকালর্ত গ মাস্টার্ি ঘ ব্যব্সা ২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ তাি ক প্রর্তপজি খ প্রভাব্ গ র্ব্চক্ষণতা ঘ কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও। র্পতৃহীন


In [191]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cuda"},   # or "cuda" if Colab GPU enabled
    encode_kwargs={"normalize_embeddings": True}
)
#intfloat/multilingual-e5-large
#sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
vectorstore = FAISS.from_texts(chunks, embedding_model)
vectorstore.save_local("bangla_vector_db")

In [193]:
vectorstore = FAISS.load_local("bangla_vector_db", embedding_model, allow_dangerous_deserialization=True)

In [194]:
vectorstore.save_local('/content/drive/MyDrive/10MS Dataset/bangla_vector_db')

In [195]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "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"
openai_api_base = "https://openrouter.ai/api/v1"

llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct:free",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=openai_api_base,
    temperature=0.2
)

In [196]:
short_term_memory = deque(maxlen=3)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [198]:
def answer_with_memory(user_input):
    short_term_memory.append(user_input)
    history = "\n".join(short_term_memory)
    docs = retriever.get_relevant_documents(user_input)
    context = "\n".join([d.page_content for d in docs])

    prompt = (
        f"Conversation history:\n{history}\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {user_input}\n"
        f"Answer in Bangla with one correct word only."
    )

    response = llm.invoke(prompt)
    return response.content

print(answer_with_memory("বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"))
#কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
#বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
#অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?

#Open router Limit reached so it's showing error

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1753574400000'}, 'provider_name': None}}, 'user_id': 'user_2tc0ZYZwXnp8p4vUD2zzDIzLQ2r'}